In [4]:
import pandas as pd
import os
import requests
from io import BytesIO
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import tensorflow as tf

# Ruta base para imágenes locales
LOCAL_IMAGE_PATH = "../DataSets/cats_vs_dogs/"

# Leer el archivo CSV
csv_path = "./dogs-no-dogs.csv"
data = pd.read_csv(csv_path)

# Procesar las etiquetas como un vector binario
def preprocess_labels(row):
    #return [row['direccion'], row['fachada'], row['envio'], row['etiqueta']]
    return [row['perro'], row['gato']]

# Descargar o cargar la imagen
def load_image(row, target_size=(224, 224)):
    try:
        if pd.notna(row['urlAbsoluta']):
            # Descargar la imagen desde la URL
            response = requests.get(row['urlAbsoluta'], timeout=5)
            if response.status_code == 200:
                image = load_img(BytesIO(response.content), target_size=target_size)
            else:
                return None
        else:
            # Cargar la imagen localmente
            prefix = ''
            if "nodogs" in row['filename']:
                prefix = 'no_dogs'
            else:
                prefix = 'dogs'

            directory = os.path.join(LOCAL_IMAGE_PATH, prefix)
            print(directory, '-->', row['filename']) 
            local_path = os.path.join(directory, row['filename'])
            if os.path.exists(local_path):
                image = load_img(local_path, target_size=target_size)
            else:
                return None
        return img_to_array(image) / 255.0  # Escalar al rango [0, 1]
    except Exception as e:
        print(f"Error al cargar la imagen: {row['filename']}. Error: {e}")
        return None

# Preparar los datos
images, labels = [], []

for _, row in data.iterrows():
    image = load_image(row)
    if image is not None:
        images.append(image)
        labels.append(preprocess_labels(row))

images = np.array(images)
labels = np.array(labels)

# Dividir en conjuntos de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

# Calcular los pesos de las clases
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(labels.flatten()),
    y=labels.flatten()
)

# Convertir a diccionario para Keras
class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}

# Crear el modelo MobileNet v3
base_model = MobileNetV3Large(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
base_model.trainable = False  # Congelar el modelo base

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(4, activation='sigmoid')  # Salida para clasificación multietiqueta
])

# Compilar el modelo
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',  # Para multietiqueta
    metrics=['accuracy']
)

# Aumentación de datos para el conjunto de entrenamiento
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

# Aplicar aumentación solo a las imágenes de entrenamiento
train_generator = datagen.flow(X_train, y_train, batch_size=32)

# Entrenar el modelo
history = model.fit(
    train_generator,
    validation_data=(X_val, y_val),
    epochs=10,
    class_weight=class_weights_dict  # Aplicar los pesos de las clases
)

# Evaluar el modelo
loss, accuracy = model.evaluate(X_val, y_val)
print(f"Pérdida: {loss:.4f}, Precisión: {accuracy:.4f}")

# Guardar el modelo
model.save("multilabel_classifier_with_balancing.h5")


../DataSets/cats_vs_dogs/dogs --> 0.jpg
../DataSets/cats_vs_dogs/dogs --> 1.jpg
../DataSets/cats_vs_dogs/dogs --> 10.jpg
../DataSets/cats_vs_dogs/dogs --> 100.jpg
../DataSets/cats_vs_dogs/dogs --> 1000.jpg
../DataSets/cats_vs_dogs/dogs --> 10000.jpg
../DataSets/cats_vs_dogs/dogs --> 10001.jpg
../DataSets/cats_vs_dogs/dogs --> 10002.jpg
../DataSets/cats_vs_dogs/dogs --> 10003.jpg
../DataSets/cats_vs_dogs/dogs --> 10004.jpg
../DataSets/cats_vs_dogs/dogs --> 10005.jpg
../DataSets/cats_vs_dogs/dogs --> 10006.jpg
../DataSets/cats_vs_dogs/dogs --> 10007.jpg
../DataSets/cats_vs_dogs/dogs --> 10008.jpg
../DataSets/cats_vs_dogs/dogs --> 10009.jpg
../DataSets/cats_vs_dogs/dogs --> 1001.jpg
../DataSets/cats_vs_dogs/dogs --> 10010.jpg
../DataSets/cats_vs_dogs/dogs --> 10011.jpg
../DataSets/cats_vs_dogs/dogs --> 10012.jpg
../DataSets/cats_vs_dogs/dogs --> 10013.jpg
../DataSets/cats_vs_dogs/dogs --> 10014.jpg
../DataSets/cats_vs_dogs/dogs --> 10015.jpg
../DataSets/cats_vs_dogs/dogs --> 10016.jpg
../

C:\Users\USUARIO\anaconda3\envs\classifier\Lib\site-packages\PIL\TiffImagePlugin.py:935: UserWarning: Truncated File Read
  warnings.warn(str(msg))


../DataSets/cats_vs_dogs/dogs --> 9069.jpg
../DataSets/cats_vs_dogs/dogs --> 907.jpg
../DataSets/cats_vs_dogs/dogs --> 9070.jpg
../DataSets/cats_vs_dogs/dogs --> 9071.jpg
../DataSets/cats_vs_dogs/dogs --> 9072.jpg
../DataSets/cats_vs_dogs/dogs --> 9073.jpg
../DataSets/cats_vs_dogs/dogs --> 9074.jpg
../DataSets/cats_vs_dogs/dogs --> 9075.jpg
../DataSets/cats_vs_dogs/dogs --> 9076.jpg
../DataSets/cats_vs_dogs/dogs --> 9077.jpg
../DataSets/cats_vs_dogs/dogs --> 9078.jpg
../DataSets/cats_vs_dogs/dogs --> 9079.jpg
../DataSets/cats_vs_dogs/dogs --> 908.jpg
../DataSets/cats_vs_dogs/dogs --> 9080.jpg
../DataSets/cats_vs_dogs/dogs --> 9081.jpg
../DataSets/cats_vs_dogs/dogs --> 9082.jpg
../DataSets/cats_vs_dogs/dogs --> 9083.jpg
../DataSets/cats_vs_dogs/dogs --> 9084.jpg
../DataSets/cats_vs_dogs/dogs --> 9085.jpg
../DataSets/cats_vs_dogs/dogs --> 9086.jpg
../DataSets/cats_vs_dogs/dogs --> 9087.jpg
../DataSets/cats_vs_dogs/dogs --> 9088.jpg
../DataSets/cats_vs_dogs/dogs --> 9089.jpg
../DataSets/c

C:\Users\USUARIO\anaconda3\envs\classifier\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 2), output.shape=(None, 4)